In [1]:
import os

In [2]:
%pwd

'c:\\Users\\SRIRAM SRIPADA\\Downloads\\Credit_Churn_Prediction_with_MLFlow\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\SRIRAM SRIPADA\\Downloads\\Credit_Churn_Prediction_with_MLFlow'

## Update the entity

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    max_depth: int
    min_samples_leaf: int
    n_estimators: int
    target_column: str

## Update the configuration manager in src config

In [6]:
from src.ChurnPrediction.constants import *
from src.ChurnPrediction.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.RandomForestClassifier
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            max_depth = params.max_depth,
            min_samples_leaf = params.min_samples_leaf,
            n_estimators = params.n_estimators,
            target_column = schema.name
            
        )

        return model_trainer_config

In [8]:
import pandas as pd
import os
from src.ChurnPrediction import logger
from sklearn.ensemble import RandomForestClassifier
import joblib

In [11]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)


        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        #convert your target variable to a 1D array using the ravel()
        train_y = train_data[self.config.target_column].values.ravel()
        test_y = test_data[self.config.target_column].values.ravel()

        

        rf = RandomForestClassifier(max_depth=self.config.max_depth, min_samples_leaf=self.config.min_samples_leaf, n_estimators = self.config.n_estimators, random_state=42)
        rf.fit(train_x, train_y)

        joblib.dump(rf, os.path.join(self.config.root_dir, self.config.model_name))

In [12]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2023-12-03 01:26:44,742: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-03 01:26:44,745: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-03 01:26:44,749: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-12-03 01:26:44,751: INFO: common: created directory at: artifacts]
[2023-12-03 01:26:44,754: INFO: common: created directory at: artifacts/model_trainer]
